In [22]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.express as px
pd.set_option('display.float_format','{:.2f}'.format)

In [23]:
# Loading metrics
df_metrics = pd.read_csv("metrics.csv")

In [24]:
df_metrics.head()

,datetime,search_config,query_set_id,query,metric,value
0,2024-07-01,research_1,sampled_queries,00g 目立たない,dcg,1.63
1,2024-07-01,research_1,sampled_queries,1 & 1/4 pvp water suction hose without couplin...,dcg,0.12
2,2024-07-01,research_1,sampled_queries,10 and 8 yr old. make up without the nieal plush,dcg,1.63
3,2024-07-01,research_1,sampled_queries,11' x 11' patio umbrella square without led li...,dcg,1.08
4,2024-07-01,research_1,sampled_queries,23x10.5 tractor intertubes,dcg,1.00


In [25]:
df_metrics.datetime.size

36960

In [26]:
pd.crosstab([df_metrics.query_set_id, df_metrics.search_config], df_metrics.metric, df_metrics.value, aggfunc="mean")

metric                         dcg  jaccard  ndcg  prec@1  prec@10  prec@5
query_set_id    search_config                                             
sampled_queries baseline      2.46     1.00  0.47    0.64     0.46    0.64
                challenger_1  2.79     0.28  0.53    0.65     0.53    0.65
                research_1    0.77     0.07  0.14    0.52     0.10    0.52
                research_2    2.03     0.21  0.40    0.65     0.36    0.65
top_queries     baseline      3.05     1.00  0.51    0.63     0.60    0.63
                challenger_1  3.51     0.27  0.59    0.82     0.66    0.82
                research_1    0.91     0.05  0.14    0.54     0.13    0.54
                research_2    2.59     0.21  0.44    0.82     0.45    0.82

# Visualizations

## Visualization of single configuration

In [28]:
sel = df_metrics.search_config.isin(["research_1"])
sel = sel & (df_metrics.metric.isin(["prec@1", "dcg"]))
sel = sel & (df_metrics.query_set_id=="sampled_queries")
cols = ["search_config", "datetime", "metric"]
df = df_metrics[sel][cols + ["value"]].groupby(cols).mean().reset_index()

fig = px.line(df, x="datetime", y="value", facet_row="metric", color="metric")
fig.update_yaxes(matches=None)
fig

## Evaluating multiple search configurations

In [29]:
sel = df_metrics.search_config.isin(["research_1", "research_2", "challenger_1"])
sel = sel & (df_metrics.metric.isin(["prec@1", "dcg"]))
sel = sel & (df_metrics.query_set_id=="sampled_queries")
cols = ["search_config", "datetime", "metric"]
df = df_metrics[sel][cols + ["value"]].groupby(cols).mean().reset_index()

fig = px.line(df, x="datetime", y="value", color="search_config", facet_row="metric")
fig.update_yaxes(matches=None)
fig

In [30]:
sel = df_metrics.search_config.isin(["research_1", "research_2", "challenger_1"])
sel = sel & (df_metrics.metric.isin(["jaccard"]))
sel = sel & (df_metrics.query_set_id=="sampled_queries")

df = df_metrics[sel]
df = df[["value", "metric", "search_config", "query", "datetime"]]

fig = px.violin(df, y="value", x="search_config", points="all", color="metric", hover_data=df.columns)
fig

In [31]:
sel = df_metrics.search_config.isin(["research_1", "research_2", "challenger_1"])
sel = sel & (df_metrics.metric.isin(["dcg"]))
sel = sel & (df_metrics.query_set_id=="sampled_queries")

df = df_metrics[sel]
df = df[["value", "metric", "search_config", "query", "datetime"]]

fig = px.violin(df, y="value", x="search_config", points="all", color="metric", hover_data=df.columns)
fig

## Comparing two search configurations

In [36]:
sel = df_metrics.search_config.isin(["baseline", "challenger_1"])
sel = sel & (df_metrics.metric.isin(["dcg"]))
sel = sel & (df_metrics.query_set_id=="sampled_queries")

df = df_metrics[sel]
cols = ["metric", "search_config", "query"]
df = df[cols + ["value"]].groupby(cols).mean().reset_index()
df["value"] = df.value.apply(lambda x: round(x, 2)) # manual formatting

fig = px.violin(df, y="value", x="search_config", points="all", color="metric", hover_data=df.columns)
fig